# Проект: Исследование для платформы чтения книг

# Описание проекта

Компания приобрела сервис по чтению книг по подписке. Необходимо провести анализ имеющихся данных

Описание данных

Таблица books

Содержит данные о книгах:

- book_id — идентификатор книги;
- author_id — идентификатор автора;
- title — название книги;
- num_pages — количество страниц;
- publication_date — дата публикации книги;
- publisher_id — идентификатор издателя.

Таблица authors

Содержит данные об авторах:

- author_id — идентификатор автора;
- author — имя автора.

Таблица publishers

- Содержит данные об издательствах:
- publisher_id — идентификатор издательства;
- publisher — название издательства;

Таблица ratings

Содержит данные о пользовательских оценках книг:

- rating_id — идентификатор оценки;
- book_id — идентификатор книги;
- username — имя пользователя, оставившего оценку;
- rating — оценка книги.

Таблица reviews

Содержит данные о пользовательских обзорах:

- review_id — идентификатор обзора;
- book_id — идентификатор книги;
- username — имя автора обзора;
- text — текст обзора.

## Подготовка и ознакомление с таблицами

In [1]:
# импортируем библиотеки
import pandas as pd
import sqlalchemy as sa

In [ ]:
# устанавливаем параметры
db_config = {
    'user': 'praktikum_student',  # имя пользователя
    'pwd': 'Sdf4$2;d-d30pp',      # пароль
    'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
    'port': 6432,                 # порт подключения
    'db': 'data-analyst-final-project-db'  # название базы данных
}

connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# создаем коннектор
engine = sa.create_engine(connection_string, connect_args={'sslmode': 'require'})

# функция для выполнения SQL-запросов
def get_sql_data(query: str, engine: sa.engine.base.Engine = engine) -> pd.DataFrame:
    '''Открываем соединение, получаем данные из SQL, закрываем соединение'''
    with engine.connect() as con:
        return pd.read_sql(sql=sa.text(query), con=con)

# SQL-запрос
query = '''
SELECT * 
FROM books 
LIMIT 5
'''

# Вызов функции и вывод результата
result = get_sql_data(query)


In [ ]:
result

In [ ]:
# SQL-запросы для исследования таблиц
tables = ["books", "authors", "publishers", "ratings", "reviews"]

for table in tables:
    display(f"Таблица: {table}")
    display(get_sql_data(f"SELECT * FROM {table} LIMIT 5"))  # Первые строки
    display(get_sql_data(f"SELECT COUNT(*) AS count FROM {table}"))  # Кол-во строк


In [ ]:
books = '''
SELECT 
    *
FROM books
'''
books = get_sql_data(books)
books.describe()

Мы видим, что данные о тысячи книгах. В средням содержат 389 страниц

In [ ]:
authors = '''
SELECT 
    author_id,
    author
FROM authors

'''
authors = get_sql_data(authors)
authors['author_id'].nunique()

В данных 636 авторов

In [ ]:
publishers = '''
SELECT 
    *
FROM publishers
'''

publishers = get_sql_data(publishers)
publishers['publisher'].nunique()

340 издательств

In [ ]:
ratings = '''
SELECT 
    *
FROM ratings
'''
ratings = get_sql_data(ratings)
ratings['rating_id'].nunique()

6456 поставленных оценок 

In [ ]:
reviews = '''
SELECT 
    *
FROM reviews
'''
reviews = get_sql_data(reviews)
reviews['review_id'].nunique()

2793 пользовательских обзора

## Анализ данных

### Посчитать количество книг, вышедших после 1 января 2000 года

In [ ]:
query = '''
SELECT COUNT(*)
FROM books
WHERE publication_date > '2000-01-01';
'''
result = get_sql_data(query)
result

819 книг вышло после 1 января

### Для каждой книги посчитаем количество обзоров и среднюю оценку

In [ ]:
query = '''
SELECT 
    b.book_id,
    b.title,
    COUNT(DISTINCT r.review_id) AS review_count,
    AVG(rt.rating) AS avg_rating
FROM books b
LEFT JOIN reviews r ON b.book_id = r.book_id
LEFT JOIN ratings rt ON b.book_id = rt.book_id
GROUP BY b.book_id, b.title
ORDER BY b.title;

'''
result = get_sql_data(query)
result


Для 1000 книг, средний рейтинг 3,9. Максимальное количество обзоров 7, медианное 3.

### Определить издательство, выпустившее наибольшее число книг толще 50 страниц

In [ ]:
query = '''
SELECT 
    p.publisher,
    COUNT(*) AS books_count
FROM books b
JOIN publishers p ON b.publisher_id = p.publisher_id
WHERE b.num_pages > 50
GROUP BY p.publisher
ORDER BY books_count DESC
LIMIT 1;

'''
result = get_sql_data(query)
result


Издательство Penguin Books выпустило больше всего книг (от 50 страниц) = 42

### Найти автора с самой высокой средней оценкой книг (учитывать книги с 50 и более оценками)

In [ ]:
query = '''
WITH book_ratings AS (
   SELECT
        b.book_id as book_50,
        COUNT(rt.rating) as count_rating
    FROM books b
    JOIN
    ratings rt ON rt.book_id = b.book_id
    GROUP BY b.book_id
    HAVING COUNT(rt.rating) >=50
) 

SELECT 
    a.author,
    AVG(r.rating) as rating_avg
FROM authors a
JOIN
books b ON a.author_id = b.author_id
JOIN
ratings r ON r.book_id = b.book_id
JOIN
book_ratings br ON br.book_50 = b.book_id
GROUP BY a.author
ORDER BY rating_avg DESC
LIMIT 1
 
'''


result = get_sql_data(query)
result

J.K. Rowling/Mary GrandPré - автор с самой высокой средней оценкой книг (учитывать книги с 50 и более оценками)

### Посчитать среднее количество обзоров от пользователей, оставивших более 48 оценок

In [ ]:
query = '''
WITH active_users AS (
    SELECT 
        username
    FROM ratings
    GROUP BY username
    HAVING COUNT(rating) > 48
)
SELECT 
    AVG(review_count) AS avg_reviews_per_user
FROM (
    SELECT 
        u.username,
        COUNT(r.review_id) AS review_count
    FROM active_users u
    JOIN reviews r ON u.username = r.username
    GROUP BY u.username
) user_reviews;

'''
result = get_sql_data(query)
result

24 - среднее количество обзоров от пользователей, оставивших более 48 оценок

## Вывод

Выводы
 - Общие данные:
В таблицах содержатся данные о тысяче книгах. Среднее количество страниц в книгах составляет 389.

 - Данные об авторах:
Всего в данных представлено 636 авторов.

- Издательства:
В базе данных числится 340 издательств, среди которых издательство Penguin Books выпустило наибольшее количество книг объемом более 50 страниц — 42 книги.

- Оценки и обзоры:

Всего пользователи оставили 6456 оценок для книг.\
Также было написано 2793 пользовательских обзора.\
Книги после 2000 года:\
После 1 января 2000 года вышло 819 книг.\

Средний рейтинг и обзоры:

Средний рейтинг для 1000 книг составляет 3.9.\
Максимальное количество обзоров для одной книги — 7, медианное значение — 3.\

Лучший автор:
Автор с самой высокой средней оценкой книг (учитывая только книги с 50 и более оценками) — J.K. Rowling/Mary GrandPré.\

Активные пользователи:
Среднее количество обзоров от пользователей, оставивших более 48 оценок, составляет 24.\